In [163]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.callbacks import EarlyStopping
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

### Pipeline psuedo code
    n = look back window
    k = number of PCs to keep

    for each time point t:
        p = number of stocks in investable universe at time t
        Define an n x p feature matrix X (lagged returns)

        Perform PCA on X
        Keep the first k PCs in an n x k matrix Z

    for each stock s in the investable universe at time t:
        Define an n x 1 outcome vector y (future returns of stock s)
        Perform a linear regression of y on Z
        Predict y for stock s at time t+1

## Data Preparation

In [4]:
returns = pd.read_pickle("./Data/returns.pkl")
returns = returns.iloc[1:]

In [5]:
drop_columns = []

for col in returns.columns:
    if returns[col].isnull().all() == True:
        drop_columns.append(col)
        
returns.drop(columns=drop_columns, inplace=True)

## Pipeline

In [124]:
def get_investable(t, n):
    """"Find stocks in investable universe at time t
    (stocks in the S&P500 that have prices recorded for the last n days)"""
    
    df_investable = returns.copy(deep = True).sort_index(ascending = False)
    
    #add 1 date to get the test features in investable
    t = t + pd.DateOffset(1)
    n += 1
    
    #if t is now a non-trading day, advance until we reach a valid trading day
    while t not in df_investable.index:
        t = t + pd.DateOffset(1)
    
    t_index = df_investable.index.get_loc(t)
    
    #take n_rows worth of data upto time specified
    df_investable = df_investable.iloc[t_index + 1:t_index + n + 1]
    
    #find all stocks that exist in the S&P at this time period
    investable_universe = []
    for col in df_investable.columns:
        if ~df_investable[col].iloc[:n].isna().any():
            investable_universe.append(col)
        
    df_investable = df_investable[investable_universe]
    
    return df_investable

In [145]:
def apply_PCA(inv, k):
    pca = PCA(n_components = k) 
    inv_scaled = StandardScaler().fit_transform(inv)   
    principal_components = pca.fit_transform(inv_scaled)

    df = pd.DataFrame(data = principal_components)
    
    #For explained variance table
    components = pca.components_
    component_explained_var = pca.explained_variance_ratio_ * 100
    
    comp_names = ['PCA' + str(i) for i in range(1, len(component_explained_var) + 1)]

    pca_results = pd.DataFrame(data = component_explained_var, index = comp_names)
    pca_results.columns = ['Explained variance (%)']
    pca_results['Explained variance (%)'] = pca_results['Explained variance (%)'].round(2)
    
    return df

In [174]:
def define_y(inv, stock, n):
    y = inv[[stock]].iloc[:n+1]
    
    return y

In [160]:
def train_test(X, y):
    X_train = X.iloc[1:, :]
    X_test = X.iloc[0:1, :]
    y_train = y.iloc[1:]
    y_test = y.iloc[0:1]
    
    return X_train, y_train, X_test, y_test

In [158]:
def model_fit(X_train, y_train):
    model = sklearn.linear_model.LinearRegression()
    model.fit(X_train, y_train)
    
    return model

In [183]:
def model_predict(model, X_test):
    yhat = model.predict(X_test)
    
    return yhat

In [208]:
def predict_returns(t, n, k):
    inv = get_investable(t, n)
    X = apply_PCA(inv, k)
    
    returns_t = pd.DataFrame(index = inv.columns, columns = ['Pred', 'Actual'])
    
    for stock in inv.columns:
        y = define_y(inv, stock, n)
        X_train, y_train, X_test, y_test = train_test(X, y)
        model = model_fit(X_train, y_train)
        yhat = model_predict(model, X_test, y_test)[0][0]
        returns_t['Pred'].loc[stock] = yhat*100
        returns_t['Actual'].loc[stock] = y_test.values[0][0]
    
    return returns_t

In [214]:
def rank_stocks(returns, num_stocks):
    pred_returns = returns.sort_values(by = 'Pred', ascending = False)
    topn = pred_returns.head(num_stocks)
    botn = pred_returns.tail(num_stocks)
    
    return topn, botn

In [211]:
def portfolio_return(topn, botn, returns):
    return_t = topn['Actual'].mean() - botn['Actual'].mean()
    
    return return_t

In [242]:
def pipeline(n, k, num_stocks):

    time_range = returns.loc['2007':'2021'].index[:-1]
    
    portfolio = pd.DataFrame(index = time_range, columns = ['Portfolio Return'])
    
    count = 0
    for t in time_range:
        pred_actual = predict_returns(t, n, k)
        topn, botn = rank_stocks(pred_actual, num_stocks)
        return_t = portfolio_return(topn, botn, pred_actual)
        portfolio['Portfolio Return'].loc[t] = return_t
        count +=1
        print(f'{(count/len(time_range))*100:.2f}% complete')
    
    return portfolio

In [ ]:
portfolio = pipeline(200, 20, 5)

0.03% complete
0.06% complete
0.08% complete
0.11% complete
0.14% complete
0.17% complete
0.19% complete
0.22% complete
0.25% complete
0.28% complete
0.30% complete
0.33% complete
0.36% complete
0.39% complete
0.41% complete
0.44% complete
0.47% complete
0.50% complete
0.52% complete
0.55% complete
0.58% complete
0.61% complete
0.63% complete
0.66% complete
0.69% complete
0.72% complete
0.74% complete
0.77% complete
0.80% complete
0.83% complete
0.85% complete
0.88% complete
0.91% complete
0.94% complete
0.97% complete
0.99% complete
1.02% complete
1.05% complete
1.08% complete
1.10% complete
1.13% complete
1.16% complete
1.19% complete
1.21% complete
1.24% complete
1.27% complete
1.30% complete
1.32% complete
1.35% complete
1.38% complete
1.41% complete
1.43% complete
1.46% complete
1.49% complete
1.52% complete
1.54% complete
1.57% complete
1.60% complete
1.63% complete
1.65% complete
1.68% complete
1.71% complete
1.74% complete
1.77% complete
1.79% complete
1.82% complete
1.85% comp

14.78% complete
14.81% complete
14.84% complete
14.86% complete
14.89% complete
14.92% complete
14.95% complete
14.98% complete
15.00% complete
15.03% complete
15.06% complete
15.09% complete
15.11% complete
15.14% complete
15.17% complete
15.20% complete
15.22% complete
15.25% complete
15.28% complete
15.31% complete
15.33% complete
15.36% complete
15.39% complete
15.42% complete
15.44% complete
15.47% complete
15.50% complete
15.53% complete
15.55% complete
15.58% complete
15.61% complete
15.64% complete
15.66% complete
15.69% complete
15.72% complete
15.75% complete
15.77% complete
15.80% complete
15.83% complete
15.86% complete
15.89% complete
15.91% complete
15.94% complete
15.97% complete
16.00% complete
16.02% complete
16.05% complete
16.08% complete
16.11% complete
16.13% complete
16.16% complete
16.19% complete
16.22% complete
16.24% complete
16.27% complete
16.30% complete
16.33% complete
16.35% complete
16.38% complete
16.41% complete
16.44% complete
16.46% complete
16.49% c

28.93% complete
28.96% complete
28.99% complete
29.01% complete
29.04% complete
29.07% complete
29.10% complete
29.12% complete
29.15% complete
29.18% complete
29.21% complete
29.23% complete
29.26% complete
29.29% complete
29.32% complete
29.34% complete
29.37% complete
29.40% complete
29.43% complete
29.45% complete
29.48% complete
29.51% complete
29.54% complete
29.56% complete
29.59% complete
29.62% complete
29.65% complete
29.67% complete
29.70% complete
29.73% complete
29.76% complete
29.78% complete
29.81% complete
29.84% complete
29.87% complete
29.90% complete
29.92% complete
29.95% complete
29.98% complete
30.01% complete
30.03% complete
30.06% complete
30.09% complete
30.12% complete
30.14% complete
30.17% complete
30.20% complete
30.23% complete
30.25% complete
30.28% complete
30.31% complete
30.34% complete
30.36% complete
30.39% complete
30.42% complete
30.45% complete
30.47% complete
30.50% complete
30.53% complete
30.56% complete
30.58% complete
30.61% complete
30.64% c

43.08% complete
43.11% complete
43.13% complete
43.16% complete
43.19% complete
43.22% complete
43.24% complete
43.27% complete
43.30% complete
43.33% complete
43.35% complete
43.38% complete
43.41% complete
43.44% complete
43.46% complete
43.49% complete
43.52% complete
43.55% complete
43.57% complete
43.60% complete
43.63% complete
43.66% complete
43.68% complete
43.71% complete
43.74% complete
43.77% complete
43.79% complete
43.82% complete
43.85% complete
43.88% complete
43.91% complete
43.93% complete
43.96% complete
43.99% complete
44.02% complete
44.04% complete
44.07% complete
44.10% complete
44.13% complete
44.15% complete
44.18% complete
44.21% complete
44.24% complete
44.26% complete
44.29% complete
44.32% complete
44.35% complete
44.37% complete
44.40% complete
44.43% complete
44.46% complete
44.48% complete
44.51% complete
44.54% complete
44.57% complete
44.59% complete
44.62% complete
44.65% complete
44.68% complete
44.70% complete
44.73% complete
44.76% complete
44.79% c